In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import os
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import torch.optim as optim


class SatelliteDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace(".tif", "_FGT.tif"))
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return image, mask

In [2]:
# 데이터 경로 설정
image_dir = "C:/Users/s_zmfldlwx/Desktop/2024-1학기/OSSP-1/팀 프로젝트/New_Sample/2.원천데이터/1.항공사진_Fine_512픽셀"
mask_dir = "C:/Users/s_zmfldlwx/Desktop/2024-1학기/OSSP-1/팀 프로젝트/New_Sample/1.라벨링데이터/1.항공사진_Fine_512픽셀/1.Ground_Truth_Tiff"

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

# 데이터셋과 데이터로더 생성
dataset = SatelliteDataset(image_dir, mask_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [3]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        def CBR2d(in_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=True):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, bias=bias),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            )
        
        def ConvLayer2d(in_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=True):
            return nn.Sequential(
                CBR2d(in_channels, out_channels, kernel_size, padding, stride, bias),
                CBR2d(out_channels, out_channels, kernel_size, padding, stride, bias)
            )
        
        def PoolLayer2d(kernel_size=2, stride=2):
            return nn.MaxPool2d(kernel_size=kernel_size, stride=stride)

        def UpConvLayer2d(in_channels, out_channels, kernel_size=2, stride=2, padding=0):
            return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)

        self.encoder1 = ConvLayer2d(in_channels, 64)
        self.pool1 = PoolLayer2d(2, 2)

        self.encoder2 = ConvLayer2d(64, 128)
        self.pool2 = PoolLayer2d(2, 2)

        self.encoder3 = ConvLayer2d(128, 256)
        self.pool3 = PoolLayer2d(2, 2)

        self.encoder4 = ConvLayer2d(256, 512)
        self.pool4 = PoolLayer2d(2, 2)

        self.bridge = ConvLayer2d(512, 1024)

        self.upconv4 = UpConvLayer2d(1024, 512)
        self.decoder4 = ConvLayer2d(1024, 512)

        self.upconv3 = UpConvLayer2d(512, 256)
        self.decoder3 = ConvLayer2d(512, 256)

        self.upconv2 = UpConvLayer2d(256, 128)
        self.decoder2 = ConvLayer2d(256, 128)

        self.upconv1 = UpConvLayer2d(128, 64)
        self.decoder1 = ConvLayer2d(128, 64)

        self.output_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.encoder1(x)
        p1 = self.pool1(e1)

        e2 = self.encoder2(p1)
        p2 = self.pool2(e2)

        e3 = self.encoder3(p2)
        p3 = self.pool3(e3)

        e4 = self.encoder4(p3)
        p4 = self.pool4(e4)

        bridge = self.bridge(p4)

        u4 = self.upconv4(bridge)
        c4 = torch.cat((u4, e4), dim=1)
        d4 = self.decoder4(c4)

        u3 = self.upconv3(d4)
        c3 = torch.cat((u3, e3), dim=1)
        d3 = self.decoder3(c3)

        u2 = self.upconv2(d3)
        c2 = torch.cat((u2, e2), dim=1)
        d2 = self.decoder2(c2)

        u1 = self.upconv1(d2)
        c1 = torch.cat((u1, e1), dim=1)
        d1 = self.decoder1(c1)

        output = self.output_conv(d1)

        return output

In [ ]:
# 모델 생성
in_channels = 3  # RGB 위성 사진
out_channels = 9  # 9개 클래스

# GPU가 사용 가능한지 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 모델 생성 및 GPU로 이동
model = UNet(in_channels=3, out_channels=9).to(device)

all_masks = []
for _, mask in dataset:
    all_masks.append(mask.numpy().flatten())

all_masks = np.concatenate(all_masks)
unique, counts = np.unique(all_masks, return_counts=True)
print("Class distribution in dataset:")
for u, c in zip(unique, counts):
    print(f"Class {u}: {c} pixels")

# 각 클래스의 가중치 계산
class_weights = torch.tensor([1.0 / c for c in counts], dtype=torch.float).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, masks in dataloader:
        images = images.to(device)
        masks = masks.squeeze(1).to(device)  # 차원 줄이기
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

print("Training Finished.")

# 디렉토리 확인 및 생성
save_dir = './../model_dict_save'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 모델 저장
torch.save(model.state_dict(), os.path.join(save_dir, 'unet_model4.pth'))

In [4]:
# 모델 생성
in_channels = 3  # RGB 위성 사진
out_channels = 9  # 9개 클래스

# GPU가 사용 가능한지 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 모델 생성 및 GPU로 이동
model = UNet(in_channels=3, out_channels=9).to(device)

# 데이터셋에서 마스크 가져오기
all_masks = []
for _, mask in dataset:
    all_masks.append(mask.numpy().flatten())

all_masks = np.concatenate(all_masks)
unique, counts = np.unique(all_masks, return_counts=True)
print("Class distribution in dataset:")
for u, c in zip(unique, counts):
    print(f"Class {u}: {c} pixels")

# 각 클래스의 가중치 계산
class_weights = torch.tensor([1.0 / c for c in counts], dtype=torch.float).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 100
save_dir = './../model_dict_save'

# 디렉토리 확인 및 생성
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, masks in dataloader:
        images = images.to(device)
        masks = masks.squeeze(1).to(device)  # 차원 줄이기
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")
    
    # 모델 저장
    torch.save(model.state_dict(), os.path.join(save_dir, f'unet_model_epoch_{epoch+1}.pth'))

print("Training Finished.")


Using device: cuda
Class distribution in dataset:
Class 0.03921568766236305: 3271738 pixels
Class 0.0784313753247261: 435603 pixels
Class 0.11764705926179886: 1972758 pixels
Class 0.1568627506494522: 17130 pixels
Class 0.19607843458652496: 482 pixels
Class 0.23529411852359772: 31715 pixels
Class 0.27450981736183167: 693109 pixels
Class 0.3137255012989044: 181528 pixels
Class 0.3921568691730499: 14367457 pixels


In [ ]:
# 디렉토리 확인 및 생성
save_dir = './../model_dict_save'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 모델 저장
torch.save(model.state_dict(), os.path.join(save_dir, 'unet_model2.pth'))

In [ ]:
import torch
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 학습된 모델을 로드합니다 (이미 학습된 모델 파일이 있는 경우)
model_path = './../model_dict_save/unet_model4.pth'
model = UNet(in_channels=3, out_channels=9)
model.load_state_dict(torch.load(model_path))
model.eval()

# GPU가 사용 가능한지 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 입력 이미지 로드 및 전처리
input_image_path = './test_image/LC_AP_37607047_021.tif'
input_image = Image.open(input_image_path).convert('RGB')
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])
input_tensor = transform(input_image).unsqueeze(0).to(device)  # 배치 차원 추가 및 GPU로 이동

# 입력 이미지 시각화
plt.imshow(np.transpose(input_tensor.squeeze(0).cpu().numpy(), (1, 2, 0)))
plt.title("Input Image")
plt.show()

# 모델 예측
with torch.no_grad():
    output = model(input_tensor)
    output = torch.softmax(output, dim=1)  # 소프트맥스 적용
    output = output.squeeze(0)  # 배치 차원 제거

# 소프트맥스 적용 후 출력의 범위 확인
print(f"Output range: min={output.min().item()}, max={output.max().item()}")

# 예측 결과 확인
predicted = torch.argmax(output, dim=0).cpu().numpy()  # 가장 높은 확률의 클래스 선택

# 예측된 라벨 시각화
plt.imshow(predicted, cmap='gray')
plt.title("Predicted Label")
plt.show()

# 각 클래스별 픽셀 수 확인
unique, counts = np.unique(predicted, return_counts=True)
print("Class distribution in predicted output:")
for u, c in zip(unique, counts):
    print(f"Class {u}: {c} pixels")

# 클래스 라벨을 그레이스케일 값으로 매핑
label_to_color = {
    0: 10, 1: 20, 2: 30, 3: 40, 4: 50, 5: 60, 6: 70, 7: 80, 8: 100
}
output_mapped = np.zeros_like(predicted, dtype=np.uint8)
for k, v in label_to_color.items():
    output_mapped[predicted == k] = v

# 매핑된 결과 확인
plt.imshow(output_mapped, cmap='gray')
plt.title("Mapped Output")
plt.show()

# 매핑된 결과를 이미지로 저장
output_image = Image.fromarray(output_mapped)
output_image_path = './test_image/labeled_image.tiff'
output_image.save(output_image_path)

print("라벨링 이미지가 저장되었습니다.")



In [ ]:
import torch
import matplotlib.pyplot as plt

# 샘플 이미지와 마스크 시각화
sample_image, sample_mask = dataset[23]
plt.subplot(1, 2, 1)
plt.imshow(sample_image.permute(1, 2, 0))
plt.title("Sample Image")

plt.subplot(1, 2, 2)
plt.imshow(sample_mask.squeeze(0), cmap='gray')
plt.title("Sample Mask")

plt.show()

# 모델 로드 및 장치로 이동
model_path = './../model_dict_save/unet_model2.pth'
model = UNet(in_channels=3, out_channels=9).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# 입력 이미지 배치 차원 추가 및 장치로 이동
sample_image = sample_image.unsqueeze(0).to(device)

# 모델 출력
outputs = model(sample_image)

# 소프트맥스 적용 후 클래스 예측
outputs = torch.argmax(torch.softmax(outputs, dim=1), dim=1).squeeze().cpu().numpy()

# 예측 결과 시각화
plt.imshow(outputs, cmap='gray')
plt.title("Predicted Mask")
plt.show()

# 실제 마스크와 예측 마스크 비교
plt.subplot(1, 2, 1)
plt.imshow(sample_mask.squeeze(0).cpu(), cmap='gray')
plt.title("Actual Mask")

plt.subplot(1, 2, 2)
plt.imshow(outputs, cmap='gray')
plt.title("Predicted Mask")

plt.show()



In [ ]:
import torch
import matplotlib.pyplot as plt

# 샘플 이미지와 마스크 시각화
sample_image, sample_mask = dataset[23]
plt.subplot(1, 2, 1)
plt.imshow(sample_image.permute(1, 2, 0))
plt.title("Sample Image")

plt.subplot(1, 2, 2)
plt.imshow(sample_mask.squeeze(0), cmap='gray')
plt.title("Sample Mask")

plt.show()

# 모델 로드 및 장치로 이동
model_path = './../model_dict_save/unet_model3.pth'
model = UNet(in_channels=3, out_channels=9).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# 입력 이미지 배치 차원 추가 및 장치로 이동
sample_image = sample_image.unsqueeze(0).to(device)
sample_mask = sample_mask.unsqueeze(0).to(device)  # 필요에 따라 마스크도 이동

# 모델 출력
outputs = model(sample_image)

# 출력 및 마스크 형태 확인
print("Output shape:", outputs.shape)
print("Mask shape:", sample_mask.shape)

# 손실 계산을 위해 마스크의 차원 축소
sample_mask = sample_mask.squeeze(1)  # [1, 1, 512, 512] -> [1, 512, 512]

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 손실 계산
loss = criterion(outputs, sample_mask.long())
print("Loss:", loss.item())

In [ ]:
all_masks = []
for _, mask in dataset:
    all_masks.append(mask.numpy().flatten())

all_masks = np.concatenate(all_masks)
unique, counts = np.unique(all_masks, return_counts=True)
print("Class distribution in dataset:")
for u, c in zip(unique, counts):
    print(f"Class {u}: {c} pixels")


In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# 모델 로드 및 장치로 이동
model_path = './../model_dict_save/unet_model2.pth'
model = UNet(in_channels=3, out_channels=9).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# 샘플 이미지와 마스크 불러오기 및 전처리
sample_image, sample_mask = dataset[23]
sample_image = sample_image.unsqueeze(0).to(device)

# 모델 출력
with torch.no_grad():
    outputs = model(sample_image)
    print("Raw model output shape:", outputs.shape)

    # 소프트맥스 적용하여 각 클래스의 확률 계산
    softmax_outputs = torch.softmax(outputs, dim=1)
    print("Softmax applied output shape:", softmax_outputs.shape)
    print("Softmax output min:", softmax_outputs.min().item())
    print("Softmax output max:", softmax_outputs.max().item())

    # argmax 적용하여 각 픽셀의 최종 클래스를 선택
    predicted_mask = torch.argmax(softmax_outputs, dim=1).squeeze().cpu().numpy()
    print("Predicted mask shape:", predicted_mask.shape)
    print("Predicted mask unique values:", np.unique(predicted_mask))

# 예측 결과 시각화
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(sample_mask.squeeze(0).cpu(), cmap='gray')
plt.title("Actual Mask")

plt.subplot(1, 2, 2)
plt.imshow(predicted_mask, cmap='gray')
plt.title("Predicted Mask")

plt.show()
